In [ ]:
def draw_polyline_on_img(img, polyline, color=(1,0,0), thickness=1, closed_line=True):
    """ Draws a polyline on an image.
    
    Args:
        img (ndarray, float): RGB image in range [0,1]
        line_points (ndarray, float/int): array shape (n_point, 2) with x and y coordinates of the line: [[x0, y0], [x1, y1], [x2, y2], ...]
        color (tuple, float): RGB colors of the countur line in range [0,1].  Color format: (R, G, B)
        
    Returns:
        img_with_line (ndarray, float): Image with the line draw on it (range [0,1]).
    
    """
    # convert color into 8bit opencv friendly format:
    color_8bit = (int(255*color[0]),int(255*color[1]),int(255*color[2]))
    thickness = int(thickness)
    
    # transform image to 8bit (neccassary for cv2 drawing function)
    img_8bit = (img*255).astype('uint8')
    
    # transform points into cv2 coordinate system
    polyline_cv2 = np.zeros(polyline.shape)
    polyline_cv2[:,0], polyline_cv2[:,1] = polyline[:,1], polyline[:,0]
    # transform points into cv2 friendly format
    polyline_trans = polyline_cv2.reshape((-1,1,2)) 
    # transform to integers for drawing
    polyline_trans = polyline_trans.astype('int32')
    
    # draw line on images
    img_with_line = cv2.polylines(img_8bit, [polyline_trans], closed_line, color_8bit, thickness=thickness)
    
    # back to [0,1]
    img_with_line = img_with_line/255
    return img_with_line

def draw_circle_on_img(img, center, radius, color=(1,0,0), thickness=1):
    """Draws a circle on an image.
    
    Args:
        img (ndarray, float): RGB image in range [0,1]
        center (ndarray, float/int): Center of the circle.  Note: If float => converted to int.
        radius (int/float): Radius of the circle.  Note: If float => converted to int.
        color (tuple, float): Color of the circle in range [0,1]. Color format: (R, G, B)
        thickness (int/float): Thickness of the circle line. Note: If float => converted to int.
        
    Returns:
        img_with_circle (ndarray, float): Image with the circle drawn on it (range [0,1]).
    
    """

    # transform center into opencv-friendly format (change coordinate system, change to integer, use tuple)
    center_coordinates = (int(center[1]), int(center[0]))
    
    # transform to int
    radius = int(radius)
    thickness = int(thickness)
    
    # convert color into 8bit opencv friendly format:
    color_8bit = (int(255*color[0]),int(255*color[1]),int(255*color[2]))
    
    # transform image to 8bit (neccassary for cv2 drawing function)
    img_8bit = (img*255).astype('uint8')
    
    # draw line on images
    img_with_circle = cv2.circle(img_8bit,center_coordinates,radius,color_8bit)
    
    # back to [0,1]
    img_with_circle = img_with_circle/255
    return img_with_circle


def draw_marker_on_img(img, marker_pos, color=(1,0,0), marker_size=20, marker_thickness=3):
    # to improve
    """ Draws a marker on an image.
    
    Args:
        img (ndarray, float): RGB image in range [0,1]
        marker_pos (ndarray, float/int): Center position of the marker.  Note: If float => converted to int.
        color (tuple, float): Color of the marker in range [0,1]. Color format: (R, G, B)
        marker_size: Size of the marker in px.
        
    Returns:
        img_with_marker (ndarray, float): Image with the marker drawn on it (range [0,1]).
    
    """
    
    dl = int(marker_size/2)
    dx = np.array([dl, 0])
    dy = np.array([0, dl])
    
    line_vertical = np.array([marker_pos + dx, marker_pos - dx])
    line_horizontal = np.array([marker_pos + dy, marker_pos - dy])
    
    img_with_marker = draw_polyline_on_img(img, line_vertical, color=color, thickness=marker_thickness)
    img_with_marker = draw_polyline_on_img(img_with_marker, line_horizontal, color=color, thickness=3)
    
    return img_with_marker


def plot_fractal_dimension_regression(measurements, step_sizes):
    """ Plots the stepsizes over the calculated perimeters, fits a line and plots it. (Only for visualisation.)
    
    Args:
        step_sizes_log (ndarray, float(int)): Array of the analysed step sizes in form [s0, s1, s2, ...].
        perimeters_log (ndarray, float(int)): Array of the corresponding perimeters in form [p0, p1, p2, ...].
    
    Returns:
        None
    """
    
    step_sizes_log = np.log2(step_sizes)
    measurements_log = np.log2(measurements)
    
    print("step_sizes_log")
    print(step_sizes_log)
    print("measured_sizes_log")
    print(measurements_log)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(step_sizes_log, measurements_log)
    
    print(slope)
    plt.plot(step_sizes_log, measurements_log, 'o', label='Original data')
    plt.plot(step_sizes_log, intercept + slope*step_sizes_log, 'r', label='Fitted line')
    plt.xlabel('log(Stepsize [px])')
    plt.ylabel('log(Measured [px])')
    plt.legend()
    plt.show()
    
    return

def visualize_walking_path_fractal_dimension_perimeter_method(contour, all_idx_list):
    """ Visualizes the walking path of the perimeter method to determine the fractal dimension of a contour.
    
    Args:
        contour (ndarray, int/float): Array of contourpoints describing the shape contour
            in shape [[x0, y0], [x1, y1], [x2, y2]]
        all_idx_list (list): A list of the indexes of measurements points used at the different stepsizes in shape
            [[i00, i01, ...], [i10, i11, ...], ...]. Where each sublist represents the indexes of one walk with a
            certain step size.
            
    Returns:
        None
    """

    for idx_list in all_idx_list:
        fig, ax = plt.subplots()
        plt.scatter(contour[:,0], contour[:,1], c='grey')
        plt.scatter(contour[idx_list,0], contour[idx_list,1], c='red')
        poly = plt.Polygon(contour[idx_list,:], ec="r", fill=False, linewidth=3)
        ax.add_patch(poly)
        ax.axis('equal')
        plt.show()

    return

def transform_feret_idx_2_lines(x_idx_feret, y_max_img, y_min_img=0):
    """ Transform the x indexes of the feret calipers into two lines from y_min_img to y_max_img
        for visualisation of the feret diameter.
        
        Args:
            x_idx_feret (ndarray, int): x indexes of the feret calipers in form [idx_x0, idx_x1].
            y_max_img (int): max y coordinates of the calculated lines. (Usually width of the image).
            y_min_img (int, optional): min y coordinates of the calculated lines. (Default is 0).
            
        Returns:
            line_feret_top (ndarray, int): Line showing the top feret caliper in form [[x0, y0],[x1, y1]].
            line_feret_bottom (ndarray, int): Line showing the bottom feret caliper in form [[x0, y0],[x1, y1]].
    """
    line_feret_top = np.zeros((2,2))
    line_feret_top[0,0], line_feret_top[0,1] = x_idx_feret[0], 0
    line_feret_top[1,0], line_feret_top[1,1] = x_idx_feret[0], y_max_img
    line_feret_bottom = np.zeros((2,2))
    line_feret_bottom[0,0], line_feret_bottom[0,1] = x_idx_feret[1], 0
    line_feret_bottom[1,0], line_feret_bottom[1,1] = x_idx_feret[1], y_max_img
    
    return line_feret_top, line_feret_bottom

In [ ]:
# daplha = 9
# angles = np.arange(0,180,daplha)
# feret_diameters = np.zeros(angles.shape, dtype='int64')
# idxs_feret = []
# martin_diameters = np.zeros(angles.shape, dtype='int64')
# idxs_martin = []
# nassenstein_diameters = np.zeros(angles.shape, dtype='int64')
# idxs_nassenstein = []
# max_chords = np.zeros(angles.shape, dtype='int64')
# idxs_max_chord = []
# all_chords = []

# for i, angle in enumerate(angles):
#     print("Rotated by {}°".format(angle))
#     bw_rotated = transform.rotate(bw, angle, resize=True)
#     bw_rotated = bw_rotated > 0.5
    
#     img_rotated = transform.rotate(img, angle, resize=True)
#     # Feret diameter
#     feret_diameters[i], idx_feret = calc_feret_diameter(bw_rotated)
#     idxs_feret.append(idx_feret)
    
#     # Martin diameter
#     martin_diameters[i], idx_martin = calc_martin_diameter(bw_rotated)
#     idxs_martin.append(idx_martin)
    
#     # Nassenstein diameter
#     nassenstein_diameters[i], idx_nassenstein = calc_nassenstein_diameter(bw_rotated)
#     idxs_nassenstein.append(idx_nassenstein)
    
#     # Chords
#     chords, edgepoints = calc_chords(bw_rotated)
#     max_chords[i], idx_max_chord = determine_max_chord(chords, edgepoints)
#     all_chords.append(chords)
#     idxs_max_chord.append(idx_max_chord)    
    
#     # PLOTTING
#     # Longest chord
#     plt.figure(figsize=(17,15))
#     plt.subplot(141)
#     plt.title('Laengste Sehne: {:0.1f} px'.format(max_chords[i]))
#     img_with_chord = draw_polyline_on_img(img_rotated, idx_max_chord, thickness=3, color=(0,1,1))
#     plt.imshow(img_with_chord)

#     # Feret diameter
#     plt.subplot(142)
#     line_feret_top, line_feret_bottom = transform_feret_idx_2_lines(idx_feret, bw_rotated.shape[1])
#     img_with_feret = draw_polyline_on_img(img_rotated, line_feret_top, thickness=2, color=(1,1,0))
#     img_with_feret = draw_polyline_on_img(img_with_feret, line_feret_bottom, thickness=2, color=(1,1,0))
#     plt.imshow(img_with_feret)
#     plt.title('Feret-Durchmesser: {:0.1f} px'.format(feret_diameters[i]))

#     # Martin diameter
#     plt.subplot(143)
#     plt.title('Martin-Durchmesser: {:0.1f} px'.format(martin_diameters[i]))
#     img_with_martin = draw_polyline_on_img(img_rotated, idx_martin, thickness=3, color=(1,0,1))
#     plt.imshow(img_with_martin)

#     # Nassenstein Diameter
#     plt.subplot(144)
#     plt.title('Nassenstein-Durchmesser: {:0.1f} px'.format(nassenstein_diameters[i]))
#     img_with_nassenstein = draw_polyline_on_img(img_rotated, idx_nassenstein, thickness=3, color=(0,1,0))
#     plt.imshow(img_with_nassenstein)
#     plt.show()